<center><h1>Tuwaiq Academy</h1></center>
<center><h2>Data Science and Machine Learning Bootcamp</h2></center>
<center><h2>Usecase 8 - Project 5: Niceone Makeup</h2></center>

## Data Collected

 Data collected from the Nice One website on March 7, 2025 
 a leading Saudi e-commerce platform specializing in beauty, care, and perfume products

 
  Objectives:
> 1. Explore the dataset to uncover trends in pricing, ratings.
>  2. Perform feature engineering to enhance the dataset for modeling.
>  3. Build predictive models to analyze patterns.
>  4. Provide actionable insights that can help optimize business strategies.

### Import the libraries

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver  
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


### Web Scraping Product URLs from Nice One Makeup Section Using Selenium

In [ ]:
#Initialize the WebDriver
driver = webdriver.Chrome()

#Initialize an empty list to store the URLs
url_list = []

#Base URL for the first page
url = "https://niceonesa.com/ar/makeup"

#Loop through pages from page 1 to 118
for i in range(2, 119):
#Construct the URL for each page
    driver.get(url)


    # Extract the URLs from the page
    elements = driver.find_elements(By.XPATH, "//a[contains(@class, 'min-w-[160px]') and contains(@class, 'max-w-[450px]') and contains(@class, 'rounded-md') and contains(@class, 'bg-white')]")

    for element in elements:
        product_url = element.get_attribute("href")
        if product_url:  # Ensure there's a valid URL
            url_list.append(product_url)

    # Update the URL to the next page
    url = f"https://niceonesa.com/ar/makeup?page={i}"

#Print the list of all URLs
print(url_list)

#Close the WebDriver
driver.quit()


### Web Scraping and Data Extraction for Makeup Products Using Selenium and BeautifulSoup

#### First try for data collected

In [87]:
# Initialize the DataFrame
columns_ = ['name', 'brand_name', 'original_price', 'discounted_price', 
            'rating_number', 'reviews_number', 'makeup_type', 'skin_type', 'texture']
df = pd.DataFrame(columns=columns_)


In [98]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Load the list of URLs
url_prodct = pd.read_csv('/Users/munirahalzuman/Downloads/url_list.csv', header=None)
urls = url_prodct[0].tolist()  # Assuming the URLs are in the first column

# Initialize the WebDriver
driver = webdriver.Chrome()



# Function to check URL status
def is_url_accessible(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return True
    except requests.RequestException:
        return False
    return False

# List to store inaccessible URLs
inaccessible_urls = []

# Iterate through each URL
for url in urls:
    # Check if the URL is accessible
    if not is_url_accessible(url):
        print(f"Inaccessible URL: {url}")
        inaccessible_urls.append(url)
        continue

    # Attempt to load the URL in the browser
    try:
        driver.get(url)
        time.sleep(3)  # Wait for the page to load
    except Exception:
        print(f"Failed to load URL in browser: {url}")
        inaccessible_urls.append(url)
        continue

    # Extract rating number
    try:
        rating_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//span[contains(@class, "text-color-[#f5c523]")]'))
        )
        rating_number = rating_element.text.strip()
    except Exception:
        rating_number = 'Not Available'

    # Extract reviews number
    try:
        review_element = driver.find_element(By.XPATH, '//span[contains(text(), "التقييمات")]')
        reviews_text = review_element.text.strip()
        reviews_number = ''.join(filter(str.isdigit, reviews_text))
    except Exception:
        reviews_number = 'Not Available'

    # Click the "مواصفات" tab
    try:
        specifications_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//span[@class="van-tab__text" and contains(text(), "مواصفات")]'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", specifications_tab)
        specifications_tab.click()
    except Exception:
        makeup_type_value, skin_type_value, texture_value = 'Not Available', 'Not Available', 'Not Available'
    else:
        try:
            makeup_type_value = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(text(), "نوع المكياج")]/following-sibling::div'))
            ).text.strip()
        except Exception:
            makeup_type_value = 'Not Available'

        try:
            skin_type_value = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(text(), "نوع البشرة")]/following-sibling::div'))
            ).text.strip()
        except Exception:
            skin_type_value = 'Not Available'

        try:
            texture_value = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(text(), "القوام")]/following-sibling::div'))
            ).text.strip()
        except Exception:
            texture_value = 'Not Available'

    # Parse the page using BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Extract name
    name = soup.find('h1', {'class': "text-xl mb-4"}).text.strip() if soup.find('h1', {'class': "text-xl mb-4"}) else 'Not Available'

    # Extract brand name
    brand_name = soup.find('a', {'class': "font-niceone-medium text-blue-purple ltr:ml-1 rtl:mr-1"})
    Brand_name = brand_name.text.strip() if brand_name else 'Not Available'

    # Extract price information
    price_div = soup.find('div', class_='flex items-end')
    original_price = price_div.find('span').text.strip() if price_div else 'Not Available'

    # Check for discounted price
    discounted_price = '0'  # Default discounted price if not found
    discount_div = soup.find('div', class_='flex md:flex-row flex-col custom-container bg-white mb-5 p-8')
    if discount_div:
        discounted_price_div = discount_div.find('div', class_='flex items-center text-discount-color font-niceone-medium me-1 whitespace-nowrap')
        if discounted_price_div:
            discounted_price = discounted_price_div.find('span').text.strip() if discounted_price_div.find('span') else 'Not Available'

    # Create a new DataFrame row
    new_row = pd.DataFrame([{
        'name': name,
        'brand_name': Brand_name,
        'original_price': original_price,
        'discounted_price': discounted_price,
        'rating_number': rating_number,
        'reviews_number': reviews_number,
        'makeup_type': makeup_type_value,
        'skin_type': skin_type_value,
        'texture': texture_value
    }])

    # Append the new row to the main DataFrame
    df = pd.concat([df, new_row], ignore_index=True)

# Save the DataFrame to a CSV file
df.to_csv('/Users/munirahalzuman/Documents/product_data.csv', index=False)

# Save inaccessible URLs to a separate file
with open('inaccessible_urls.txt', 'w') as f:
    for url in inaccessible_urls:
        f.write(url + '\n')

# Quit the driver
driver.quit()

Inaccessible URL: https://niceonesa.com/ar/topface-baked-choice-rich-touch-blush-on-001-n11878
Inaccessible URL: https://niceonesa.com/ar/hearts-blush-powder-11-g-n33743
Inaccessible URL: https://niceonesa.com/ar/topface-waterproof-lipliner-113-n13876
Inaccessible URL: https://niceonesa.com/ar/carissa-cosmetics-all-in-one-brow-kit-3-pieces-n23338
Inaccessible URL: https://niceonesa.com/ar/note-cosmetique-perfecting-pen-concealer-n32394
Inaccessible URL: https://niceonesa.com/ar/topface-focus-point-matte-lip-paint-009-n10053
Inaccessible URL: https://niceonesa.com/ar/make-over-22-shimmer-eyeliner-m3009-n21331
Inaccessible URL: https://niceonesa.com/ar/topface-instyle-twin-blush-blush-on-006-n20936
Inaccessible URL: https://niceonesa.com/ar/relove-baby-gloss-lip-gloss-n27448
Inaccessible URL: https://niceonesa.com/ar/bourjois-rouge-velvet-ink-liquid-lipstick-n26178-12395
Inaccessible URL: https://niceonesa.com/ar/make-over-22-liquid-eye-shadow-m2613-n16365
Inaccessible URL: https://niceo

In [ ]:
# Convert the list of inaccessible URLs into a DataFrame with a column named 'inaccessible_urls'
inaccessible_urls =pd.DataFrame({'inaccessible_urls': inaccessible_urls})

In [112]:
# Save the set of inaccessible URLs to a CSV file
inaccessible_urls.to_csv('/Users/munirahalzuman/Documents/inaccessible_urls.csv', index=False)

#### Second Attempt at Data Collection for Inaccessible URLs from the First Try


In [6]:
# Initialize the DataFrame
columns_ = ['name', 'brand_name', 'original_price', 'discounted_price', 
            'rating_number', 'reviews_number', 'makeup_type', 'skin_type', 'texture']
df2 = pd.DataFrame(columns=columns_)

In [7]:
# second try for data collection for accessible_urls

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Load the list of URLs
url_prodct = pd.read_csv('/Users/munirahalzuman/Documents/inaccessible_urls.csv')
urls = url_prodct['inaccessible_urls'].tolist()   # Access the column by name

# Initialize the WebDriver
driver = webdriver.Chrome()



# Function to check URL status
def is_url_accessible(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return True
    except requests.RequestException:
        return False
    return False

# List to store inaccessible URLs
inaccessible_urls_2 = []

# Iterate through each URL
for url in urls:
    # Check if the URL is accessible
    if not is_url_accessible(url):
        print(f"Inaccessible URL: {url}")
        inaccessible_urls_2.append(url)
        continue

    # Attempt to load the URL in the browser
    try:
        driver.get(url)
        time.sleep(3)  # Wait for the page to load
    except Exception:
        print(f"Failed to load URL in browser: {url}")
        inaccessible_urls_2.append(url)
        continue

    # Extract rating number
    try:
        rating_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//span[contains(@class, "text-color-[#f5c523]")]'))
        )
        rating_number = rating_element.text.strip()
    except Exception:
        rating_number = 'Not Available'

    # Extract reviews number
    try:
        review_element = driver.find_element(By.XPATH, '//span[contains(text(), "التقييمات")]')
        reviews_text = review_element.text.strip()
        reviews_number = ''.join(filter(str.isdigit, reviews_text))
    except Exception:
        reviews_number = 'Not Available'

    # Click the "مواصفات" tab
    try:
        specifications_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//span[@class="van-tab__text" and contains(text(), "مواصفات")]'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", specifications_tab)
        specifications_tab.click()
    except Exception:
        makeup_type_value, skin_type_value, texture_value = 'Not Available', 'Not Available', 'Not Available'
    else:
        try:
            makeup_type_value = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(text(), "نوع المكياج")]/following-sibling::div'))
            ).text.strip()
        except Exception:
            makeup_type_value = 'Not Available'

        try:
            skin_type_value = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(text(), "نوع البشرة")]/following-sibling::div'))
            ).text.strip()
        except Exception:
            skin_type_value = 'Not Available'

        try:
            texture_value = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(text(), "القوام")]/following-sibling::div'))
            ).text.strip()
        except Exception:
            texture_value = 'Not Available'

    # Parse the page using BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Extract name
    name = soup.find('h1', {'class': "text-xl mb-4"}).text.strip() if soup.find('h1', {'class': "text-xl mb-4"}) else 'Not Available'

    # Extract brand name
    brand_name = soup.find('a', {'class': "font-niceone-medium text-blue-purple ltr:ml-1 rtl:mr-1"})
    Brand_name = brand_name.text.strip() if brand_name else 'Not Available'

    # Extract price information
    price_div = soup.find('div', class_='flex items-end')
    original_price = price_div.find('span').text.strip() if price_div else 'Not Available'

    # Check for discounted price
    discounted_price = '0'  # Default discounted price if not found
    discount_div = soup.find('div', class_='flex md:flex-row flex-col custom-container bg-white mb-5 p-8')
    if discount_div:
        discounted_price_div = discount_div.find('div', class_='flex items-center text-discount-color font-niceone-medium me-1 whitespace-nowrap')
        if discounted_price_div:
            discounted_price = discounted_price_div.find('span').text.strip() if discounted_price_div.find('span') else 'Not Available'

    # Create a new DataFrame row
    new_row = pd.DataFrame([{
        'name': name,
        'brand_name': Brand_name,
        'original_price': original_price,
        'discounted_price': discounted_price,
        'rating_number': rating_number,
        'reviews_number': reviews_number,
        'makeup_type': makeup_type_value,
        'skin_type': skin_type_value,
        'texture': texture_value
    }])

    # Append the new row to the main DataFrame
    df2 = pd.concat([df2, new_row], ignore_index=True)

# Save the DataFrame to a CSV file
df2.to_csv('/Users/munirahalzuman/Documents/product_data_part2.csv', index=False)

# Save inaccessible URLs to a separate file
with open('inaccessible_urls_2.txt', 'w') as f:
    for url in inaccessible_urls_2:
        f.write(url + '\n')

# Quit the driver
driver.quit()

Inaccessible URL: https://niceonesa.com/ar/topface-supernova-volume-lipgloss-010-n22828
Inaccessible URL: https://niceonesa.com/ar/elf-high-definition-loose-powder-sheer-n29
Inaccessible URL: https://niceonesa.com/ar/mesauda-cult-matte-lipstick-211-sexysweet-n22086
Inaccessible URL: https://niceonesa.com/ar/makeup-eyes-mascara-bourjois-volume-glamour-lift-stretch-mascara-black-n26499
Inaccessible URL: https://niceonesa.com/ar/ofra-pro-palette-soul-n11717
Inaccessible URL: https://niceonesa.com/ar/kahi-kisstin-balm-pink-n27068
Inaccessible URL: https://niceonesa.com/ar/makeup-brushes-tools-makeup-tools-inglot-freedom-system-gloss-palette-n26862
Inaccessible URL: https://niceonesa.com/ar/essence-fix-last-14h-makeup-fixing-loose-powder-n24108
Inaccessible URL: https://niceonesa.com/ar/benefit-hoola-matte-bronzer-powder-n683
Inaccessible URL: https://niceonesa.com/ar/kiko-velvet-touch-creamy-stick-blush-006-geranium-n23633
Inaccessible URL: https://niceonesa.com/ar/kiko-green-me-kajal-eyel

In [11]:
len(inaccessible_urls_2)

43

In [13]:
# Convert the list of inaccessible URLs into a DataFrame with a column named 'inaccessible_urls_2'
inaccessible_urls_2_file =pd.DataFrame({'inaccessible_urls': inaccessible_urls_2})

In [14]:
# Save the second set of inaccessible URLs to a CSV file
inaccessible_urls_2_file.to_csv('/Users/munirahalzuman/Documents/len(inaccessible_urls_2.csv', index=False)